In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.path import Path
from aocd.models import Puzzle

# Day 1

In [2]:
def fuel(mass):
    return np.floor(mass/3)-2

def recursive_fuel(mass):
    total_fuel = 0
    while np.any(mass > 0):
        mass = fuel(mass)
        total_fuel += np.maximum(mass, 0)
    return total_fuel      

In [3]:
# test
fuel(12) == 2.0
fuel(14) == 2.0
fuel(1969) == 654.0
fuel(100756) == 33583.0
recursive_fuel(14) == 2.0
recursive_fuel(1969) == 966
recursive_fuel(100756) == 50346.0

p = Puzzle(year=2019, day=1)
module_mass = np.array(p.input_data.split('\n'), dtype='int')

p.answer_a = int(np.sum(fuel(module_mass)))
p.answer_b = int(np.sum(recursive_fuel(module_mass)))

print('Total full needed for all module is %s.' % p.answer_a)
print('Total full (and the fuel of fuel) needed for all module is %s.' % p.answer_b)

Total full needed for all module is 3538016.
Total full (and the fuel of fuel) needed for all module is 5304147.


# Day 2

In [4]:
def run_intcode(code):
    ptr = 0
    opcode = code[ptr]
    while opcode != 99:
        param1 = code[ptr+1]
        param2 = code[ptr+2]
        out  = code[ptr+3]
        
        # addition
        if opcode == 1:
            code[out] = code[param1] + code[param2]
        
        # multiplication
        elif opcode == 2:
            code[out] = code[param1] * code[param2]
        
        else:
            print('Error while processing.')
            return code
        
        # move pointer
        ptr += 4
        opcode = code[ptr]
    return code

In [5]:
p = Puzzle(year=2019, day=2)
intcode = list(map(int, p.input_data.split(',')))

# part a: reset to the 1202 program alarm
intcode_i = np.copy(intcode)
intcode_i[1] = 12
intcode_i[2] = 2
out = run_intcode(intcode_i)
p.answer_a = out[0]
print('The value left at address 0 is %s.' % p.answer_a)

# part b : search parameters that give a specific result
find = False
result = 19690720
for noun in range(0, 100):
    for verb in range(0, 100):
        intcode_i = np.copy(intcode)
        intcode_i[1] = noun
        intcode_i[2] = verb
        out = run_intcode(intcode_i)
        if out[0] == result:
            p.answer_b = (100 * noun + verb)
            find = True
            break   
    if find: 
        break
print('The answer of: 100 * noun + verb = %s.' % p.answer_b)

Part a already solved with same answer: 4714701
The value left at address 0 is 4714701.
The answer of: 100 * noun + verb = 5121.


# Day 3

In [6]:
# dictionarry with keys (x,y) of each point 
# and value the number step from the beginning
def create_line(moves):
    l = {}
    
    # current position
    x, y = 0,0
    step = 0
    for move in moves:
        direction = move[0]
        steps = int(move[1:])
        
        # set move in both direction
        move_x, move_y = 0,0
        if direction == 'U':
            move_y = 1
        elif direction == 'D':
            move_y = -1
        elif direction == 'R':
            move_x = 1
        elif direction == 'L':
            move_x = -1
        else:
            print('Wrong direction')
            return l
        
        for _ in range(0, steps):
            x += move_x
            y += move_y
            step += 1
            
            if (x,y) not in l:  # don't have to store doubles
                l[(x,y)] = step
    return l

def closest_crossing(intersections):
    d = []
    for i in intersections:
        d.append(abs(i[0]) + abs(i[1]))
    return min(d)

def min_combined_steps(l1, l2, intersection):
    steps = []
    for i in intersections:
        steps.append(l1[i] + l2[i])
    return min(steps)

In [7]:
p = Puzzle(2019, 3)
data = p.input_data.split('\n')

In [8]:
raw_move_1 = data[0].split(',')
raw_move_2 = data[1].split(',')

l1 = create_line(raw_move_1)
l2 = create_line(raw_move_2)

# Get the intersections for both lines
intersections = l1.keys() & l2.keys()
p1 = closest_crossing(intersections)
p2 = min_combined_steps(l1, l2, intersections)

p.answer_a = p1
p.answer_b = p2
print('The closest intersection is at a distance of %d.' % p1)
print('The fewest combined steps to an intersection is %d.' % p2)

The closest intersection is at a distance of 870.
The fewest combined steps to an intersection is 13698.


# Day 4

In [9]:
def increasing_order(num):
    num_s = str(num)
    for i in range(0, len(num_s)-1):
        if num_s[i] > num_s[i+1]:
            return False
    return True

def repeating_adjacent_digit(num):
    num_s = str(num)
    for i in range(0, len(num_s)-1):
        if num_s[i] == num_s[i+1]:
            return True
    return False

def repeating_adjacent_digit_p2(num):
    digit_repeat = {}
    num_s = str(num)
    for i in range(0, len(num_s)-1):
        if num_s[i] == num_s[i+1]:
            if num_s[i] in digit_repeat:
                digit_repeat[num_s[i]] += 1
            else:
                digit_repeat[num_s[i]] = 1
                
    # has to be only 1 repetition of at least 1 digit
    for d in digit_repeat:
        if digit_repeat[d] == 1:
            return True
    return False

In [10]:
p = Puzzle(2019, 4)
min_i, max_i = map(int, p.input_data.split('-'))

In [11]:
# input is the password range 356261-846303
c = 0
for i in range(min_i, max_i):
    if increasing_order(i) & repeating_adjacent_digit(i):
        c += 1

p.answer_a = c
print('There are %d valid password in that range.' % c)

There are 544 valid password in that range.


In [12]:
# input is the password range 356261-846303

c = 0
for i in range(min_i, max_i):
    if increasing_order(i) & repeating_adjacent_digit_p2(i):
        c += 1

p.answer_b = c
print('There are %d valid password in that range.' % c)

There are 334 valid password in that range.


# Day 5

In [13]:
def param_mode(code, value, mode):
    if mode == 0:
        return code[code[value]]
    if mode == 1:
        return code[value]

def mode_and_instruction(s):
    len_ins = len(s)
    a, b, c = 0, 0, 0
    if len_ins == 4:
        c = int(s[1])
        b = int(s[0])
        opcode = int(s[2:])
    elif len_ins == 3:
        c = int(s[0])
        opcode = int(s[2:])
    elif len_ins == 2:
        opcode = int(s)
    else:
        opcode = int(s)
    return a, b, c, opcode
    
def run_intcode_v2(code, input_code):
    ptr = 0
    opcode_str = str(code[ptr])
    while opcode_str != '99':  # exit opcode
        
        # parse mode and instruction
        a, b, c, opcode = mode_and_instruction(opcode_str)
        
        # addition
        if opcode == 1:
            param1 = param_mode(code, ptr+1, c)
            param2 = param_mode(code, ptr+2, b)
            param3 = code[ptr+3]
            code[param3] = param1 + param2
            move_ptr = 4
        
        # multiplication
        elif opcode == 2:
            param1 = param_mode(code, ptr+1, c)
            param2 = param_mode(code, ptr+2, b)
            param3 = code[ptr+3]
            code[param3] = param1 * param2
            move_ptr = 4
        
        # save input parameter to position
        elif opcode == 3:
            param1 = code[ptr+1]
            code[param1] = input_code
            move_ptr = 2
        
        # output the only parameter
        elif opcode == 4:
            param1 = param_mode(code, ptr+1, c)
            print('$ Diagnostic code %d.' % param1)
            move_ptr = 2
            
        # jump-if-true
        elif opcode == 5:
            param1 = param_mode(code, ptr+1, c)
            param2 = param_mode(code, ptr+2, b)
            if param1:
                ptr = param2
                move_ptr = 0
            else:
                move_ptr = 3
        
        # jump-if-false
        elif opcode == 6:
            param1 = param_mode(code, ptr+1, c)
            param2 = param_mode(code, ptr+2, b)
            if param1 == 0:
                ptr = param2
                move_ptr = 0
            else:
                move_ptr = 3
        
        # if less than
        elif opcode == 7:
            param1 = param_mode(code, ptr+1, c)
            param2 = param_mode(code, ptr+2, b)
            param3 = code[ptr+3]
            if param1 < param2:
                code[param3] = 1
            else:
                code[param3] = 0
            move_ptr = 4
        
        # if equals
        elif opcode == 8:
            param1 = param_mode(code, ptr+1, c)
            param2 = param_mode(code, ptr+2, b)
            param3 = code[ptr+3]
            if param1 == param2:
                code[param3] = 1
            else:
                code[param3] = 0
            move_ptr = 4
        else:
            print('Error while processing.')
            return code
        
        # move the pointer
        ptr += move_ptr
        opcode_str = str(code[ptr])
    return code, param1

In [14]:
p = Puzzle(2019, 5)
data = list(map(int, p.input_data.split(',')))

In [15]:
# part one with input: 1
intcode_i = np.copy(data)
outcode, param1 = run_intcode_v2(intcode_i, input_code=1)

p.answer_a = param1

$ Diagnostic code 0.
$ Diagnostic code 0.
$ Diagnostic code 0.
$ Diagnostic code 0.
$ Diagnostic code 0.
$ Diagnostic code 0.
$ Diagnostic code 0.
$ Diagnostic code 0.
$ Diagnostic code 0.
$ Diagnostic code 15097178.
Part a already solved with same answer: 15097178


In [16]:
# part two with input: 5
intcode_i = np.copy(data)
outcode, param1 = run_intcode_v2(intcode_i, input_code=5)

p.answer_b = param1

$ Diagnostic code 1558663.
Part b already solved with same answer: 1558663


# Day 6